In [3]:
# !pip install numpy
# !pip install pandas

import numpy as np
import pandas as pd

In [4]:
True_news = pd.read_csv('True.csv')
Fake_news = pd.read_csv('Fake.csv')

In [5]:
True_news['label'] = 0
Fake_news['label'] = 1

In [6]:
dataset1 = True_news[['text', 'label']]
dataset2 = Fake_news[['text', 'label']]

In [ ]:
dataset1 #True news

,text,label
0,WASHINGTON (Reuters) - The head of a conservat...,0
1,WASHINGTON (Reuters) - Transgender people will...,0
2,WASHINGTON (Reuters) - The special counsel inv...,0
3,WASHINGTON (Reuters) - Trump campaign adviser ...,0
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,0
...,...,...
995,WASHINGTON (Reuters) - A U.S. congressional co...,0
996,WASHINGTON (Reuters) - U.S. President Donald T...,0
997,WASHINGTON (Reuters) - U.S. Secretary of State...,0
998,WASHINGTON (Reuters) - Former CIA director Jam...,0


In [ ]:
dataset2 #Fake news

,text,label
0,Donald Trump just couldn t wish all Americans ...,1
1,House Intelligence Committee Chairman Devin Nu...,1
2,"On Friday, it was revealed that former Milwauk...",1
3,"On Christmas day, Donald Trump announced that ...",1
4,Pope Francis used his annual Christmas Day mes...,1
...,...,...
995,"Not long ago, Rep. Jason Chaffetz (R-Utah), sa...",1
996,"According to a new report, health insurance pr...",1
997,The Congressional Budget Office released their...,1
998,Donald Trump was in a room full of Irish repor...,1


In [9]:
combined_df=pd.concat([dataset1 ,dataset2]) #combine the two datasets using concat function

In [10]:
combined_df

,text,label
0,WASHINGTON (Reuters) - The head of a conservat...,0
1,WASHINGTON (Reuters) - Transgender people will...,0
2,WASHINGTON (Reuters) - The special counsel inv...,0
3,WASHINGTON (Reuters) - Trump campaign adviser ...,0
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,0
...,...,...
995,"Not long ago, Rep. Jason Chaffetz (R-Utah), sa...",1
996,"According to a new report, health insurance pr...",1
997,The Congressional Budget Office released their...,1
998,Donald Trump was in a room full of Irish repor...,1


In [11]:
combined_df.shape

(2000, 2)

In [14]:
combined_df.isnull().sum() #check for missing values

text     0
label    0
dtype: int64

In [ ]:
# !pip install scipy
# scipy is a library used for scientific and technical computing
from scipy import stats

label_mean = combined_df['label'].mean()
label_median = combined_df['label'].median()
label_mode_result = stats.mode(combined_df['label']) 

# Access mode and its count directly without indexing
label_mode = label_mode_result.mode
label_mode_count = label_mode_result.count

print(f"Mean: {label_mean}")
print(f"Median: {label_median}")
print(f"Mode: {label_mode} (occurs {label_mode_count} times)")


Mean: 0.5
Median: 0.5
Mode: 0 (occurs 1000 times)


In [18]:
# Perform ANOVA F-test
f_statistic, p_value = stats.f_oneway(combined_df[combined_df['label'] == 0]['label'],
 combined_df[combined_df['label'] == 1]['label'])
print(f"F-statistic: {f_statistic}")
print(f"P-value: {p_value}")

F-statistic: inf
P-value: 0.0


C:\Users\gg490\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\scipy\stats\_axis_nan_policy.py:586: ConstantInputWarning: Each of the input arrays is constant; the F statistic is not defined or infinite
  res = hypotest_fun_out(*samples, **kwds)


In [19]:
# Create a new column 'text_length' with the length of the 'text' column
combined_df['text_length'] = combined_df['text'].apply(lambda x: len(x.split()))
# Separate text lengths for the two groups
true_text_length = combined_df[combined_df['label'] == 0]['text_length']
fake_text_length = combined_df[combined_df['label'] == 1]['text_length']


In [21]:
combined_df 

,text,label,text_length
0,WASHINGTON (Reuters) - The head of a conservat...,0,749
1,WASHINGTON (Reuters) - Transgender people will...,0,624
2,WASHINGTON (Reuters) - The special counsel inv...,0,457
3,WASHINGTON (Reuters) - Trump campaign adviser ...,0,376
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,0,852
...,...,...,...
995,"Not long ago, Rep. Jason Chaffetz (R-Utah), sa...",1,460
996,"According to a new report, health insurance pr...",1,311
997,The Congressional Budget Office released their...,1,372
998,Donald Trump was in a room full of Irish repor...,1,333


In [20]:
# Perform an independent t-test
t_statistic, p_value = stats.ttest_ind(true_text_length, fake_text_length)
print(f"T-statistic: {t_statistic}")
print(f"P-value: {p_value}")

T-statistic: -2.5639058981394967
P-value: 0.01042271414679376


In [22]:
from scipy.stats import chi2_contingency
# Replace "Your_Categorical_Variable" with the actual categorical variable name
categorical_variable ='label'
# Create a contingency table to assess the association between 'label' and the categorical variable
contingency_table = pd.crosstab(combined_df['label'], combined_df[categorical_variable])
# Perform a chi-squared test
chi2, p, _, _ = chi2_contingency(contingency_table)
print(f"Chi-squared statistic: {chi2}")
print(f"P-value: {p}")


Chi-squared statistic: 1996.002
P-value: 0.0


In [25]:
# !pip install statsmodels
# statsmodels is a library for estimating and interpreting statistical models

import statsmodels.api as sm
# Specify the independent variable (label) and dependent variable (word count)
X = combined_df[['label']] # Independent variable
y = combined_df['text_length'] # Dependent variable
# Add a constant term to the independent variable (intercept)
X = sm.add_constant(X)
# Fit a linear regression model
model = sm.OLS(y, X).fit()
# Get the regression summary
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:            text_length   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     6.574
Date:                Mon, 17 Feb 2025   Prob (F-statistic):             0.0104
Time:                        23:08:22   Log-Likelihood:                -13564.
No. Observations:                2000   AIC:                         2.713e+04
Df Residuals:                    1998   BIC:                         2.714e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        393.4860      6.751     58.282      0.0

In [27]:
# !pip install scikit-learn
# scikit-learn is a library for machine learning and statistical modeling

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# Specify the independent variables (features) and the target variable (label)
X = combined_df[['text_length']] # Replace with your feature columns
y = combined_df['label'] # Target variable
# Split the data into a training set and a testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
random_state=42)
# Initialize and fit the logistic regression model
logistic_regression = LogisticRegression()
logistic_regression.fit(X_train, y_train)
# Make predictions on the testing set
y_pred = logistic_regression.predict(X_test)
# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(f"Confusion Matrix:\n{confusion}")
print(f"Classification Report:\n{classification_rep}")

Accuracy: 0.52
Confusion Matrix:
[[125  74]
 [118  83]]
Classification Report:
              precision    recall  f1-score   support

           0       0.51      0.63      0.57       199
           1       0.53      0.41      0.46       201

    accuracy                           0.52       400
   macro avg       0.52      0.52      0.51       400
weighted avg       0.52      0.52      0.51       400



In [28]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
# Initialize and fit the decision tree classifier
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)
# Make predictions on the testing set
y_pred = decision_tree.predict(X_test)
# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(f"Confusion Matrix:\n{confusion}")
print(f"Classification Report:\n{classification_rep}")

Accuracy: 0.6825
Confusion Matrix:
[[132  67]
 [ 60 141]]
Classification Report:
              precision    recall  f1-score   support

           0       0.69      0.66      0.68       199
           1       0.68      0.70      0.69       201

    accuracy                           0.68       400
   macro avg       0.68      0.68      0.68       400
weighted avg       0.68      0.68      0.68       400



In [29]:
from sklearn.ensemble import RandomForestClassifier
# Initialize and fit the Random Forest classifier
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest.fit(X_train, y_train)
# Make predictions on the testing set
y_pred = random_forest.predict(X_test)
# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(f"Confusion Matrix:\n{confusion}")
print(f"Classification Report:\n{classification_rep}")

Accuracy: 0.7025
Confusion Matrix:
[[128  71]
 [ 48 153]]
Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.64      0.68       199
           1       0.68      0.76      0.72       201

    accuracy                           0.70       400
   macro avg       0.71      0.70      0.70       400
weighted avg       0.71      0.70      0.70       400



In [30]:
from sklearn.neighbors import KNeighborsClassifier
# Initialize and fit the K-NN classifier
knn = KNeighborsClassifier(n_neighbors=5) # You can adjust the number of neighbors (k) as needed
knn.fit(X_train, y_train)
# Make predictions on the testing set
y_pred = knn.predict(X_test)
# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(f"Confusion Matrix:\n{confusion}")
print(f"Classification Report:\n{classification_rep}")

Accuracy: 0.6975
Confusion Matrix:
[[123  76]
 [ 45 156]]
Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.62      0.67       199
           1       0.67      0.78      0.72       201

    accuracy                           0.70       400
   macro avg       0.70      0.70      0.70       400
weighted avg       0.70      0.70      0.70       400



In [31]:
from sklearn.svm import SVC
# Initialize and fit the SVM classifier
svm_classifier = SVC(kernel='linear') # You can choose different kernels such as 'linear', 'rbf', or 'poly'
svm_classifier.fit(X_train, y_train)
# Make predictions on the testing set
y_pred = svm_classifier.predict(X_test)
# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(f"Confusion Matrix:\n{confusion}")
print(f"Classification Report:\n{classification_rep}")

Accuracy: 0.555
Confusion Matrix:
[[100  99]
 [ 79 122]]
Classification Report:
              precision    recall  f1-score   support

           0       0.56      0.50      0.53       199
           1       0.55      0.61      0.58       201

    accuracy                           0.56       400
   macro avg       0.56      0.55      0.55       400
weighted avg       0.56      0.56      0.55       400



In [ ]:
import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# Build the ANN model
max_words=10000
max_sequence_length=10000
model = keras.Sequential([
 keras.layers.Embedding(input_dim=max_words, output_dim=16, 
input_length=max_sequence_length),
 keras.layers.Flatten(),
 keras.layers.Dense(64, activation='relu'),
 keras.layers.Dense(1, activation='sigmoid')
])
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, 
random_state=42)
# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_val, 
y_val))
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy}")


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

# Now you can use TfidfVectorizer in your code
combined_text = pd.concat([dataset1['text'], dataset2['text']])
# Preprocess the text data using TF-IDF vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=1000) # Adjust the number of features as needed
tfidf_matrix = tfidf_vectorizer.fit_transform(combined_text)
# Perform K-Means clustering
num_clusters = 2 # Assuming you want to separate true news and fake news
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(tfidf_matrix)
# Add cluster labels to your DataFrames
dataset1['cluster_label'] = kmeans.labels_[:len(dataset1)]
dataset2['cluster_label'] = kmeans.labels_[len(dataset1):]
# Print cluster centers (optional)
print("Cluster Centers:")
print(kmeans.cluster_centers_)


In [ ]:
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
# Preprocess the text data using TF-IDF vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=1000) # Adjust the number of features as needed
tfidf_matrix = tfidf_vectorizer.fit_transform(combined_text)
# Perform Truncated SVD (PCA alternative) to reduce dimensionality
n_components = 50 # Number of components to retain
svd = TruncatedSVD(n_components=n_components)
tfidf_svd = svd.fit_transform(tfidf_matrix)
# Perform K-Means clustering on the SVD-transformed data
num_clusters = 2 # Assuming you want to separate true news and fake news
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(tfidf_svd)
# Add cluster labels to your DataFrames
dataset1['cluster_label'] = kmeans.labels_[:len(dataset1)]
dataset2['cluster_label'] = kmeans.labels_[len(dataset1):]
# Print cluster centers (optional)
print("Cluster Centers:")
print(kmeans.cluster_centers_)


In [ ]:
from sklearn.metrics import silhouette_score
silhouette_avg = silhouette_score(tfidf_svd, kmeans.labels_)
print(f"Silhouette Score: {silhouette_avg}")

In [ ]:
inertia = kmeans.inertia_
print(f"Inertia: {inertia}")

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# Create true labels based on your domain knowledge
true_labels = [0] * len(dataset1) + [1] * len(dataset2)
accuracy = accuracy_score(true_labels, kmeans.labels_)
confusion = confusion_matrix(true_labels, kmeans.labels_)
report = classification_report(true_labels, kmeans.labels_)
print(f"Accuracy: {accuracy}")
print("Confusion Matrix:\n", confusion)
print("Classification Report:\n", report)